# Filtering log data with RePlay

This notebook presents the RePlay functionality for interactions data filtering.<br>
You can conveniently filter your datasets, for example, leave only positive interactions, select the desired period of time or number of interactions, filter out users and items with low number of interactions.
<br>RePlay offers filtering functions:

* MinCountFilter - filter out users or items with a few interactions
* LowRatingFilter - filter out low ratings/relevance values
* NumInteractionsFilter - select required number of the first/last user's interactions
* GlobalDaysFilter - select required number of days from interactions
* EntityDaysFilter - select required number of days from each entity interactions
* TimePeriodFilter - select interactions from required time period


In [ ]:
! pip install rs-datasets -q

In [2]:
import pandas as pd
from rs_datasets import MovieLens

from pyspark.sql import functions as sf
from pyspark.sql.types import TimestampType

from replay.utils.session_handler import State
from replay.utils.spark_utils import get_log_info, convert2spark, process_timestamp_column
from replay.preprocessing.filters import (
    MinCountFilter,
    LowRatingFilter,
    NumInteractionsFilter,
    EntityDaysFilter,
    GlobalDaysFilter,
    TimePeriodFilter,
)

In [ ]:
spark = State().session
spark.sparkContext.setLogLevel('ERROR')
spark

## Get started

Download the dataset **MovieLens**

In [4]:
data = MovieLens("1m")
log = data.ratings

Use the `convert2spark` function to convert pandas.DataFrame to spark.DataFrame

In [5]:
log_spark = convert2spark(log)

Convert timestamp to appropriate format

In [6]:
log_spark = process_timestamp_column(log_spark, "timestamp")

In [7]:
log_spark.show(4)

+-------+-------+------+-------------------+
|user_id|item_id|rating|          timestamp|
+-------+-------+------+-------------------+
|      1|   1193|     5|2001-01-01 01:12:40|
|      1|    661|     3|2001-01-01 01:35:09|
|      1|    914|     3|2001-01-01 01:32:48|
|      1|   3408|     4|2001-01-01 01:04:35|
+-------+-------+------+-------------------+
only showing top 4 rows



## MinCountFilter

Filter out users or items with number of interactions less than given threshold.

Parameters:

* `num_entries` - threshold for number of interactions
    
* `groupby_column` - column relative to which filtering occurs
    

The picture shows how filtering works using the example of a simple interaction log.<br>
Colored cells represent rating for given user and item. The color depends on the rating.

![title](img/filter_by_min_count.jpg)

Let's leave users who have `>= 50` interactions with items

In [8]:
log_filter = MinCountFilter(num_entries=50, groupby_column="user_id").transform(log_spark)

Check minimum number of interactions per user

In [9]:
log_filter.groupBy("user_id").count().select(sf.min("count").alias("Minimum number of interactions per user's")).show()

+-----------------------------------------+
|Minimum number of interactions per user's|
+-----------------------------------------+
|                                       50|
+-----------------------------------------+



Number of users has decreased

In [10]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 943471, total users: 4297, total items: 3689


Let's leave items that `>= 100` users interacted with

In [11]:
log_filter = MinCountFilter(num_entries=100, groupby_column="item_id").transform(log_spark)

Check minimum number of interactions per item

In [12]:
log_filter.groupBy("item_id").count().select(sf.min("count").alias("Minimum number of interactions per item's")).show()

+-----------------------------------------+
|Minimum number of interactions per item's|
+-----------------------------------------+
|                                      100|
+-----------------------------------------+



Number of items has decreased

In [13]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 942225, total users: 6040, total items: 2019


## LowRatingFilter

Filter out interactions with ratings/relevance less than given threshold.

Parameters:

* `value` - rating/relevance threshold

* `rating_column` - rating/relevance column
    

![title](img/filter_out_low_ratings.jpg)

Let's leave interactions with rating `>= 4`

In [14]:
log_filter = LowRatingFilter(value=4, rating_column="rating").transform(log_spark)

Check minimum and maximum rating

In [15]:
log_filter.select(sf.min("rating").alias("Minimum rating")).show()
log_filter.select(sf.max("rating").alias("Maximum rating")).show()

+--------------+
|Minimum rating|
+--------------+
|             4|
+--------------+

+--------------+
|Maximum rating|
+--------------+
|             5|
+--------------+



The number of users and interactions has decreased

In [16]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 575281, total users: 6038, total items: 3533


## NumInteractionsFilter

Select required number of the first/last user's interactions.

Parameters:

    
* `num_interactions` - threshold value for the number of interactions for each user
    
* `first` - indicator, if the value is `True`, take first `num_interactions` for each user, if the value is `False`, take last `num_interactions` for each user
    
* `timestamp_column` - date column name
    
* `query_column` - user id column name
    
* `item_column` - item id column name


![title](img/take_num_user_interactions.jpg)

Let's leave the last `2` interactions for each user

In [17]:
log_filter = NumInteractionsFilter(
    num_interactions=2,
    first=False,
    query_column="user_id",
    item_column="item_id",
).transform(log_spark)
                                        

Check maximum number of interactions per user

In [18]:
log_filter.groupBy("user_id").count().select(sf.max("count").alias("Maximum number of interactions per user's")).show()

+-----------------------------------------+
|Maximum number of interactions per user's|
+-----------------------------------------+
|                                        2|
+-----------------------------------------+



Note that the number of users has not changed

In [19]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 12080, total users: 6040, total items: 2430


## EntityDaysFilter

Select required number of days from each user's or item's interactions.

Parameters:
   
* `days` - time interval in days
    
* `first` - indicator, if the value is `True`, take first days of each user's interactions, if the value is `False`, take last days of each user's interactions
    
* `timestamp_column` - date column name
    
* `entity_column` - user/item id column name

![title](img/take_num_days_of_user_hist.jpg)

Let's leave the last `week` of each user's interactions

In [20]:
log_filter = EntityDaysFilter(
    days=7,
    first=False,
    entity_column="user_id",
    timestamp_column="timestamp",
).transform(log_spark)
                                        

Check maximum length of user history

In [21]:
(
    log_filter
    .groupBy("user_id")
    .agg(sf.datediff(sf.max("timestamp"), sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("Maximum difference in days per user's"))
).show()

+-------------------------------------+
|Maximum difference in days per user's|
+-------------------------------------+
|                                    7|
+-------------------------------------+



The number of users has not changed, but the number of interactions has decreased

In [22]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 551897, total users: 6040, total items: 3599


## GlobalDaysFilter

Select required number of days from interaction log.

Parameters:

    
* `days` - time interval in days
    
* `first` - indicator, if the value is `True`, take first days from log, if the value is `False`, take last days from log
    
* `timestamp_column` - date column name

![title](img/take_num_days_of_global_hist.jpg)

Let's leave the last `3 weeks` of interactions

In [23]:
log_filter = GlobalDaysFilter(days=21, first=False).transform(log_spark)

In [24]:
(
    log_filter
    .agg(sf.datediff(sf.max("timestamp"), sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("Maximum difference in days among all interactions"))
).show()

+-------------------------------------------------+
|Maximum difference in days among all interactions|
+-------------------------------------------------+
|                                               21|
+-------------------------------------------------+



In [25]:
(
    log_spark
    .select(
        sf.max("timestamp")
        .alias("Maximum date in log")
    )
).show()

(
    log_filter
    .select(
        sf.max("timestamp")
        .alias("Maximum date in filtered log")
    )
).show()

+-------------------+
|Maximum date in log|
+-------------------+
|2003-02-28 20:49:50|
+-------------------+

+----------------------------+
|Maximum date in filtered log|
+----------------------------+
|         2003-02-28 20:49:50|
+----------------------------+



After such filtering, the number of interactions and users significantly decreased

In [26]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 920, total users: 82, total items: 704


## TimePeriodFilter

Select interactions from required time period.

Parameters:

    
* `start_date` - beginning of required period
    
* `end_date` - end of required period
    
* `timestamp_column` - date column name

![title](img/take_time_period.jpg)

Let's leave the data for `March` only

In [27]:
log_filter = TimePeriodFilter(start_date="2001-03-01", end_date="2001-04-01").transform(log_spark)

Check maximum and minimum date

In [28]:
(
    log_filter
    .select(
        sf.min("timestamp").alias("Minimum date"),
        sf.max("timestamp").alias("Maximum date")
    )                            
).show()

+-------------------+-------------------+
|       Minimum date|       Maximum date|
+-------------------+-------------------+
|2001-03-01 03:08:52|2001-03-31 23:12:22|
+-------------------+-------------------+



The number of records has noticeably decreased

In [29]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 5986, total users: 321, total items: 2010
